In [ ]:
#general packages
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from collections import Counter
#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
#custom function
from false_positive_analysis import percent_false_positive
%config InlineBackend.figure_format='retina'

In [ ]:
#read in final mtx
#grab all channels
all_channels = []

for c in [1]:
    mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/genebycell/final_11p52_33_heg_svm_0p10_diff0_fdr10/final/genebycell_{c}.csv", index_col=0)
    mtx = mtx.T
    mtx = mtx[mtx.sum(axis=1)>100].T
    all_channels.append(mtx)

mtx = pd.concat(all_channels, axis=0).fillna(0)
#mtx = mtx.T
#mtx = mtx[mtx.sum(axis=1)>300].T

In [ ]:
#take a look
mtx

In [ ]:
#read in rnaseq data
rnaseq = pd.read_csv("./RNAseq_files/NIH3T3_CCS_TPM_REP1.csv")
rnaseq.columns = ["Genes","TPM"]

In [ ]:
#codebook
codebooks = ["codebook_string_647.csv","codebook_string_561.csv","codebook_string_488.csv"]
codebook = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/decoding_files/SVM_Feature_Radial_Decoding/codebook_converter/{codebooks[c-1]}", index_col=0)
#separate into true and false codebook
fakebook = codebook[codebook.index.str.startswith("fake")]
codebook = codebook.drop(fakebook.index)

In [ ]:
#calculate fdr
fp, fake = percent_false_positive(mtx, codebook, fakebook)
percent_fp = fp["FP raw"].mean()
mean_counts = fp["total_real"].mean()
sum_counts = fp["total_counts"].sum()
norm_fpr = fp["FDR"].mean()
fp_list = [percent_fp,norm_fpr,mean_counts,sum_counts]

In [ ]:
#take a look at fdr results
df_stats = pd.DataFrame(fp_list).T
df_stats.columns = ["percent fp","false positive rate","mean true counts", "total sum"]
df_stats

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
rnaseq["Genes"] = rnaseq["Genes"].str.lower()
#merge
comb_1 = pd.merge(rnaseq,bulk)
#pearson's correlation
r = pearsonr(comb_1["TPM"],comb_1["Counts"])
r = round(r[0],2)

In [ ]:
#get log2 + 1
comb_1["Log Counts"] = np.log10(comb_1["Counts"]+0.1)
comb_1["Log TPM"] = np.log10(comb_1["TPM"]+0.1)

In [ ]:
#RNA-seq plot
sns.set_style("white")
joint_kws=dict(gridsize=50)
hexplot = sns.jointplot(data=comb_1, x="Log TPM", y="Log Counts", kind="hex",mincnt=0.1, 
              cmap="plasma", dropna=True, joint_kws=joint_kws)
plt.xlabel("Bulk RNAseq Log10(TPM+0.1)", fontsize=12)
plt.ylabel("Pseudobulk Log10(Counts+0.1)", fontsize=12)
hexplot.ax_marg_x.remove()
hexplot.ax_marg_y.remove()
plt.annotate(f"Pearson's r= {r}", (-0.8,1.2), fontsize=12)
plt.title("All Channels", fontweight="bold")
plt.colorbar()
sns.despine()

In [ ]:
#read in smfish 
smfish = pd.read_csv("./nih3t3_smfish/smFISH_results.csv").T
#smfish = pd.read_csv("./nih3t3_smfish/smfish_27gene_custom_thresh_2.csv", index_col=0)
_150genes = pd.read_csv("./150genes_diff0.csv", index_col=0)

In [ ]:
#convert to pseudobulk results 
smfish_df = pd.DataFrame(smfish.mean(axis=1))
smfish_df = smfish_df.reset_index()
smfish_df.columns = ["Genes", "smFISH Counts"]
smfish_df["Genes"] = smfish_df["Genes"].str.lower()

_150genes = pd.DataFrame(_150genes.mean(axis=1))
_150genes = _150genes.reset_index()
_150genes.columns = ["Genes", "150 genes"]
_150genes["Genes"] = _150genes["Genes"].str.lower()

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
#merge
comb_2 = pd.merge(smfish_df,bulk)
#calculate info
x = comb_2["smFISH Counts"].values
x_t = np.vstack([x, np.zeros(len(x))]).T
y = comb_2["Counts"].values
m,c = np.linalg.lstsq(x_t, y, rcond=None)[0]
r = pearsonr(x,y)[0]

In [ ]:
figure(figsize=(4,5), dpi=80)
#show smfish correlation
plt.plot(x, y, 'bo', alpha=0.5)
plt.plot(x, x*m, c = "k")
plt.title("All Channels", fontweight="bold")
plt.xlabel("Average smFISH counts")
plt.ylabel("Average LANTERN Counts")
plt.annotate(f"Pearson's r= {round(r,2)}", (0,16), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (0,15), fontsize=12)
sns.despine()
plt.show()

In [ ]:
figure(figsize=(4,5), dpi=80)
#plot correlation and efficiency
plt.scatter(x = np.log2(comb_2["smFISH Counts"]), y = np.log2(comb_2["Counts"]), c="b", alpha=0.5)
plt.xlabel("Log2(Average smFISH Counts)", fontsize=12)
plt.ylabel("Log2(Average LANTERN Counts)", fontsize=12)
plt.annotate(f"Pearson's r= {round(r,2)}", (-1.2,4.2), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (-1.2,3.7), fontsize=12)
plt.title("All Channels", fontweight="bold")
sns.despine()

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
#merge
comb_2 = pd.merge(_150genes,bulk)
comb_2 = comb_2.drop(comb_2[comb_2["Genes"].str.startswith("fake")].index)
#calculate info
x = comb_2["150 genes"].values
x_t = np.vstack([x, np.zeros(len(x))]).T
y = comb_2["Counts"].values
m,c = np.linalg.lstsq(x_t, y, rcond=None)[0]
r = pearsonr(x,y)[0]

In [ ]:
figure(figsize=(4,5), dpi=80)
#show smfish correlation
plt.plot(x, y, 'bo', alpha=0.5)
plt.plot(x, x*m, c = "k")
plt.title("All Channels", fontweight="bold")
plt.xlabel("Average 150 gene LANTERN counts")
plt.ylabel("Average LANTERN Counts")
plt.annotate(f"Pearson's r= {round(r,2)}", (0,16), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (0,15), fontsize=12)
sns.despine()
plt.show()

In [ ]:
figure(figsize=(4,5), dpi=80)
#plot correlation and efficiency
plt.scatter(x = np.log2(comb_2["150 genes"]), y = np.log2(comb_2["Counts"]), c="b", alpha=0.5)
plt.xlabel("Log2(Average 150 gene LANTERN Counts)", fontsize=12)
plt.ylabel("Log2(Average LANTERN Counts)", fontsize=12)
plt.annotate(f"Pearson's r= {round(r,2)}", (-5,4.2), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (-5,3.6), fontsize=12)
plt.title("All Channels", fontweight="bold")
sns.despine()

# Percent decoded

In [ ]:
#get average percent decoded
percent_decoded_list = []
for i in range(52):
    for z in range(1):
        try:
            src = f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p10_diff0_fdr10_seed2/Channel_3/Pos_{i}/percent_decoded_z_{z}.txt"
            with open(src) as f:
                decoded = f.readlines()[0].split(" ")[-1]
                f.close()
                percent_decoded_list.append(float(decoded))
        except FileNotFoundError:
            continue

In [ ]:
np.mean(percent_decoded_list)

# Random checks

In [ ]:
#check spot count diff
fold_diff = []
channel = 2
for i in range(25):
    current = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    current = current[current["hyb"] < 12]
    old = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/1000genes_040222/notebook_pyfiles/dots_detected/Channel_1_preprocessed/genes_in_cells/Pos{i}/locations_z_0.csv")
    old = old[old["hyb"] <11]

    old_counts_cell = np.mean(old.groupby("cell number").size())
    current_counts_cell = np.mean(current.groupby("cell number").size())
    
    fold = current_counts_cell/old_counts_cell
    
    fold_diff.append(fold)

In [ ]:
np.mean(fold_diff)

In [ ]:
percent_und_list = []
channel=3
for i in range(25):
    current_und = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p10_diff0_fdr10/Channel_{channel}/Pos_{i}/dots_used_undefined_z_0.csv")
    current_und = current_und[current_und["hyb"] < 48]
    current_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    current_loc = current_loc[current_loc["hyb"] < 48]
    current_per = len(current_und)/len(current_loc)
    
    old_und = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p10_diff0_fdr10/Channel_{channel}/Pos_{i}/dots_used_undefined_z_0.csv")
    old_und = old_und[old_und["hyb"] <48]
    old_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    old_loc = old_loc[old_loc["hyb"] <48]
    old_per = len(old_und)/len(old_loc)
    
    diff = old_per - current_per
    
    percent_und_list.append(diff)

In [ ]:
np.mean(percent_und_list)*100

In [ ]:
percent_fake_list = []
channel=3
for i in range(25):
    current_fake = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p10_diff0_fdr10/Channel_{channel}/Pos_{i}/dots_used_fakes_z_0.csv")
    current_fake = current_fake[current_fake["hyb"] < 48]
    current_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/060322_4kgenes/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    current_loc = current_loc[current_loc["hyb"] < 48]
    current_per = len(current_fake)/len(current_loc)
    
    old_fake = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/decoded/final_11p52_33_heg_svm_0p10_diff0_fdr10/Channel_{channel}/Pos_{i}/dots_used_fakes_z_0.csv")
    old_fake = old_fake[old_fake["hyb"] <48]
    old_loc = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/dots_detected/Channel_{channel}/genes_in_cells/Pos{i}/locations_z_0.csv")
    old_loc = old_loc[old_loc["hyb"] <48]
    old_per = len(old_fake)/len(old_loc)
    
    diff = old_per - current_per
    
    percent_fake_list.append(diff)

In [ ]:
np.mean(percent_fake_list)*100

# Identify problematic hybs

In [ ]:
#collapse into gene counts
counts_df = Counter(fake["genes"])
#change to df
counts_df = pd.DataFrame.from_dict(counts_df, orient='index').reset_index()
counts_df

In [ ]:
#check codebook
codebook = pd.read_csv("/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/barcode_key/codebook_647nm.csv", index_col=0)

In [ ]:
#lowercase gene names
codebook.index = codebook.index.str.lower()

In [ ]:
#get top 30
maj_fakes = counts_df.sort_values(0, ascending=False).head(30)

In [ ]:
#get troubled hybs
troubled_hybs = []
for fakes in maj_fakes["index"]:
    troubled_hybs.append(codebook.loc[fakes].values.tolist())

In [ ]:
#convert to array
hybstocheck = np.array(troubled_hybs)

In [ ]:
#take a look
hybstocheck

In [ ]:
#get mode for each round
bad_hybs = []
for i in range(len(hybstocheck[0])):
    vals,counts = np.unique(hybstocheck[:,i], return_counts=True)
    index = np.argmax(counts)
    bad_hyb = vals[index]
    bad_hybs.append(bad_hyb)

print(f"Following hybs by rounds are problematic:{bad_hybs}" )